# 一摞Python风格的纸牌

Python最好的品质之一就是其编码风格的一致性，这种风格也被称为Pythonic，Pythonic就是Python的设计思想，这种设计思想完全体现在Python的数据模型上，而数据模型所描述的API，为使用最地道的语言特性来构建你自己的对象提供了工具。

数据模型其实是对Python框架的描述，它规范了这门语言自身构建模块的接口，这些模块包括但不限于序列、迭代器、函数、类和上下文管理器。

不管在那种框架下写程序，都需要去实现很多那些会被框架本身调用的方法，Python也是。Python解释器碰到特殊的句法时，会使用特殊方法去激活一些基本的对象操作，这些特殊的方法名字以两个下划线开头两个下划线结尾（例如 `__getitem__`）。

这些特殊方法能让你自己的对象实现和支持以下语言构架，并与之交互：

- 迭代
- 集合类
- 属性访问
- 运算符重载
- 函数和方法的调用
- 对象的创建和销毁
- 字符串表达形式和格式化
- 管理上下文（即 `with` 块）

下面以一个简单的例子展示如何实现 `__getitem__` 和 `__len__` 这两个魔法方法。

在该例子中，我们建立了一个包含52张纸牌的类：

In [1]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck(object):
    
    ranks = [str(i) for i in range(2, 11)] + list('JQKA')
    suits = '黑桃 方块 梅花 红心'.split()
    
    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, position):
        return self._cards[position]
    
deck = FrenchDeck()

首先我们用 `collections.namedtuple` 构建了一个简单的类来表示纸牌，`namedtuple` 一般用于构建只有少数属性且没有方法的对象。

如下面的例子中，利用它可以轻松的得到一个纸牌对象：

In [2]:
beer_card = Card('7', '黑桃')
print(beer_card)

Card(rank='7', suit='黑桃')


当然，我们这个例子中主要关注的还是 `FrenchDeck` 这个类，它跟任何Python的集合类型一样，可以用 `len()` 函数来查看一叠纸牌的张数。

In [3]:
len(deck)

52

从一叠纸牌中抽取特定的一张纸牌，比如第一张或最后一张，是很容易的：`deck[0]` 或 `deck[-1]`。

这都是由 `__getitem__` 方法提供的：

In [4]:
deck[0]

Card(rank='2', suit='黑桃')

In [5]:
deck[-1]

Card(rank='A', suit='红心')

如果需要随机的抽取一张纸牌，我们需要单独的编写一个方法或函数吗？

没必要，我们完全可以使用Python标准库中内置的 `random.choice`，直接把它用到这一摞纸牌实例上就好：

In [6]:
from random import choice

choice(deck)

Card(rank='3', suit='方块')

因为 `__getitem__` 方法把 `[]` 操作交给了 `self._cards` 列表，所以我们的 `deck` 对象自动支持切片操作。

下面以获取一摞纸牌最上面3张和只看牌面为A的纸牌的操作为例：

In [7]:
deck[:3]

[Card(rank='2', suit='黑桃'),
 Card(rank='3', suit='黑桃'),
 Card(rank='4', suit='黑桃')]

In [8]:
deck[12::13]

[Card(rank='A', suit='黑桃'),
 Card(rank='A', suit='方块'),
 Card(rank='A', suit='梅花'),
 Card(rank='A', suit='红心')]

另外，仅仅实现了 `__getitem__` 方法，这个对象就自动变成可迭代对象了：

In [9]:
for card in deck[:5]:
    print(card)
    
print('.' * 6)

Card(rank='2', suit='黑桃')
Card(rank='3', suit='黑桃')
Card(rank='4', suit='黑桃')
Card(rank='5', suit='黑桃')
Card(rank='6', suit='黑桃')
......


至于排序，我们可以按照常规的方式来判断纸牌的大小，2最小A最大；同时对于花色，黑桃最大、红心次之、方块再次、梅花最小。

下面按照这个规则来实现排序函数：

In [10]:
suit_values = {'黑桃': 3, '红心': 2, '方块': 1, '梅花': 0}

def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_values) + suit_values[card.suit]

In [11]:
for card in sorted(deck[:5], key=spades_high):
    print(card)

print('.' * 6)

Card(rank='2', suit='黑桃')
Card(rank='3', suit='黑桃')
Card(rank='4', suit='黑桃')
Card(rank='5', suit='黑桃')
Card(rank='6', suit='黑桃')
......
